## Installation

```bash
pip install dml-py
```

Or install from source:
```bash
git clone https://github.com/yourusername/DML-PY.git
cd DML-PY
pip install -e .
```

## 1. Import Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from dml-py.trainers import DMLTrainer
from dml-py.models.cifar import resnet20, wrn_16_2

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Prepare Data

Let's use CIFAR-10 dataset for this tutorial:

In [ ]:
# Data transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Download datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

## 3. Create Models

For DML, we need multiple models. They can have different architectures!

In [ ]:
# Create 2 different models
models = [
    resnet20(num_classes=10),
    wrn_16_2(num_classes=10)
]

print(f"Model 1: ResNet-20")
print(f"Model 2: WideResNet-16-2")
print(f"\nNumber of models: {len(models)}")

## 4. Train with DML

This is where the magic happens! Just initialize the trainer and call train():

In [ ]:
# Initialize DML trainer
trainer = DMLTrainer(
    models=models,
    learning_rate=0.1,
    temperature=3,
    kl_weight=1.0
)

# Train for 10 epochs
results = trainer.train(
    train_loader=train_loader,
    val_loader=test_loader,
    epochs=10,
    save_dir='dml_checkpoints'
)

print("\n✓ Training complete!")

## 5. View Results

In [ ]:
print("Final Results:")
print(f"  Model 1 Accuracy: {results['model_0_acc'][-1]:.2f}%")
print(f"  Model 2 Accuracy: {results['model_1_acc'][-1]:.2f}%")
print(f"  Average Accuracy: {results['avg_acc'][-1]:.2f}%")

## 6. Visualize Training

In [ ]:
import matplotlib.pyplot as plt

# Plot training curves
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(results['model_0_acc'], label='Model 1', linewidth=2)
plt.plot(results['model_1_acc'], label='Model 2', linewidth=2)
plt.plot(results['avg_acc'], label='Average', linewidth=2, linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Test Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss
plt.subplot(1, 2, 2)
plt.plot(results['train_loss'], label='Training Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Compare with Baseline

Let's see how much improvement DML provides:

In [ ]:
# Train single model (baseline)
from dml-py.core import BaseTrainer

baseline_model = resnet20(num_classes=10)
baseline_trainer = BaseTrainer(
    model=baseline_model,
    learning_rate=0.1
)

baseline_results = baseline_trainer.train(
    train_loader=train_loader,
    val_loader=test_loader,
    epochs=10
)

# Compare
baseline_acc = baseline_results['val_acc'][-1]
dml_acc = results['avg_acc'][-1]
improvement = dml_acc - baseline_acc

print(f"\nComparison:")
print(f"  Baseline (single model): {baseline_acc:.2f}%")
print(f"  DML (2 models average):  {dml_acc:.2f}%")
print(f"  Improvement:             +{improvement:.2f}%")
print(f"\n{'🎉 DML is better!' if improvement > 0 else '⚠️  Try more epochs'}")

## Next Steps

Now that you've mastered the basics, explore:

1. **Advanced Training** - Check out `02_advanced_training.ipynb` for:
   - Curriculum learning
   - Peer selection
   - Temperature scaling

2. **Different Methods** - Try `03_distillation_methods.ipynb` for:
   - Knowledge Distillation
   - Co-Distillation
   - Feature-based DML

3. **Visualization** - See `04_visualization.ipynb` for:
   - Loss landscape analysis
   - Knowledge flow visualization
   - Training dynamics

4. **Production** - Learn in `05_deployment.ipynb`:
   - Model export (ONNX, TorchScript)
   - Mixed precision training
   - Distributed training

## Resources

- 📖 [Documentation](https://github.com/yourusername/DML-PY)
- 💬 [Issues](https://github.com/yourusername/DML-PY/issues)
- 📝 [Examples](https://github.com/yourusername/DML-PY/tree/main/examples)

Happy learning! 🚀